In [365]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import nn
import numpy as np
import os
import yaml
from pathlib import Path
from PIL import Image

In [366]:
path = "./archive/"

In [367]:
argv = "./baseline_config.yaml"

In [368]:
config = yaml.safe_load(open(argv, "r"))

In [369]:
config

{'model': {'name': 'Baseline'}, 'augmentation': {'name': '00'}}

In [370]:
class BaseLineModel(nn.Module):
    def __init__(self):
        pass
class BestCNN(nn.Module):
    def __init__(self):
        pass
class Transformer(nn.Module):
    def __init__(self):
        pass

def create_model(config: dict):
    if config["model"]["name"] == "Baseline":
        model = BaseLineModel()
    elif config["model"]["name"] == "BestCNN":
        model = BestCNN()
    else:
        model = Transformer()
    
    return model

def get_transform():
    transform = transforms.Compose(
        [
            transforms.Resize((128, 128)),
            transforms.ToTensor(),  # Shape: HWC, Scales data into [0,1] by div / 255
            transforms.Normalize(mean=[0, 0, 0], std=[1, 1, 1]) # image_channel = (image - mean) / std
        ]
    )

    return transform

def create_dataloaders(config: dict):
    augmentations = []
    if config["augmentation"]["name"] == 'all':
        augmentations = ['00', '01', '02', '03', '04', '05', '06']
    elif not config["augmentation"]["name"]:
        augmentations = ['00']
    else:
        augmentations.append(config["augmentation"]["name"])
        
    transformation = get_transform() # get transformations on the image
    
    sea_animals_train = SeaAnimalsDataset(
                                        img_path="train_augment",
                                        transform=transformation,
                                        train=True,
                                        augmentations=augmentations)
    sea_animals_val = SeaAnimalsDataset(
                                        img_path="val",
                                        transform=transformation,
                                        train=False)
    sea_animals_test = SeaAnimalsDataset(
                                        img_path="test",
                                        transform=transformation,
                                        train=False)
    train_loader = DataLoader(sea_animals_train, batch_size=1, shuffle=True, drop_last=False)
    val_loader = DataLoader(sea_animals_val, batch_size=1, shuffle=True, drop_last=False)
    test_loader = DataLoader(sea_animals_test, batch_size=1, shuffle=True, drop_last=False)
    
    return train_loader, val_loader, test_loader
    

def make(config: dict):
    """Return model and dataloader with/without data augmentation"""
    
    model = create_model(config) # get model
    trainloader, val_loader, test_loader = create_dataloaders(config)
        
    return model, train_loader, val_loader, test_loader
        


model, train_loader, val_loader, test_loader = make(config)

In [376]:
len(train_loader), len(val_loader), len(test_loader)

(8242, 2345, 1155)

In [343]:
def walk_through_dir(path: str):
    dir_tree_generator = os.walk(path)
    dirpath, dirnames, filenames = next(dir_tree_generator)
    print("Root folder: ", dirpath)
    print("It has {} subfolders and {} images".format(len(dirnames), len(filenames)))
    print()
    
    for dirpath, dirnames, filenames in dir_tree_generator:
        print("This is a folder: ", dirpath)
        print("It has {} subfolders and {} images".format(len(dirnames), len(filenames)))
        label = os.path.basename(dirpath)
        print("Class: ", label, "\n")

In [331]:
walk_through_dir("train_augment/"), walk_through_dir("train/"), walk_through_dir("val/"), walk_through_dir("test/")

Root folder:  train_augment/
It has 0 subfolders and 57694 images

Root folder:  train/
It has 0 subfolders and 8242 images

Root folder:  val/
It has 0 subfolders and 2345 images

Root folder:  test/
It has 0 subfolders and 1155 images



(None, None, None, None)

In [378]:
def _get_name(path: str):
    name, ext = os.path.splitext(path.split("_")[-2])
    
    return name

In [319]:
from typing import List

class SeaAnimalsDataset:

    def __init__(self, img_path, transform,
                 train: bool = True, augmentations: List[str] = ['00']):
        self.img_path = img_path
        self.transform = transform
        self.total_imgs = os.listdir(self.img_path)

        if train:
            self.total_imgs = [
                img_path for img_path in self.total_imgs if img_path[-6:-4] in augmentations]
        self.total_labels = list(map(lambda img_path: _get_name(img_path), self.total_imgs))

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.img_path, self.total_imgs[idx])
        image = Image.open(img_loc).convert("RGB")
        label = self.total_labels[idx]
        out_image = self.transform(image)
        out_image = transforms.Compose([transforms.ToTensor()])(out_image)

        return out_image, label

In [347]:
augmentations = ['00']
train_imgs_path = 'train_augment'
sea_animals_train = SeaAnimalsDataset(
        img_path=train_imgs_path,
        transform=transformation,
        train=True,
        augmentations=augmentations)
sea_animals_val = SeaAnimalsDataset(
    img_path="val",
    transform=transformation,
    train=False)
sea_animals_test = SeaAnimalsDataset(
    img_path="test",
    transform=transformation,
    train=False)

In [358]:
train_loader = DataLoader(sea_animals_train, batch_size=1, shuffle=True, drop_last=False)
val_loader = DataLoader(sea_animals_val, batch_size=1, shuffle=True, drop_last=False)
test_loader = DataLoader(sea_animals_test, batch_size=1, shuffle=True, drop_last=False)

In [359]:
type(train_loader), type(val_loader), type(test_loader)

(torch.utils.data.dataloader.DataLoader,
 torch.utils.data.dataloader.DataLoader,
 torch.utils.data.dataloader.DataLoader)

In [363]:
total = sum([len(train_loader), len(val_loader), len(test_loader)])
print(total)
print(len(train_loader), len(val_loader), len(test_loader))
print(len(train_loader)/total, len(val_loader)/total, len(test_loader)/total)

11742
8242 2345 1155
0.7019247146993698 0.19971044115142225 0.09836484414920797


In [ ]:
# model_1
# plot of f1 on train-val data
# plot of f1 on train-val data with augmentations
# average f1 on hold-out data set 
# model_2
# transformer